# COMP90042 Project 2018: Question Answering

Group Name: Strong

Team member:

Yunjie Jia 864538

qiangtou Guang 123456

Huorui QIang 234567

## General info

<b>Due date</b>: Due date: 11pm, Sunday 27th May, 2018

